# Analyze sampling of the OPES Explore production simulation

## Preparation

In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

### Imports

In [ ]:
# standard library
import logging
import os
from pathlib import Path    
import sys

# third-party packages
import cmasher as cmr
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
from scipy import stats, integrate
from tqdm.notebook import tqdm

# notebook
from IPython import display

# get absolute path to file's parent directory
dir_proj_base = Path(os.getcwd()).resolve().parents[1]
sys.path.insert(0, f"{dir_proj_base}/src")

# Internal dependencies
from render.figure_style import set_style  # noqa: E402
from data.pipeline import DataPipeline  # noqa: E402


### Logging and formatting

In [ ]:
# notebook logger has DEBUG level
log = logging.getLogger("notebook")
log.setLevel(logging.DEBUG)

# console handler
format = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
ch = logging.StreamHandler(sys.stdout)
ch.setFormatter(format)
log.addHandler(ch)

In [ ]:
# set plotting style
set_style()

## Parameters

### System

Data input from the production simulation

In [ ]:
# paths
sim_tag = Path("completed/sjobid_0-calcite-104surface-5nm_surface-8nm_vertical-1chain-PAcr-8mer-0Crb-0Ca-8Na-0Cl-300K-1bar-NVT")
data_dir_base = Path(
    f"/media/aglisman/Linux_Overflow/home/aglisman/VSCodeProjects/Polyelectrolyte-Surface-Adsorption/data"
) / sim_tag


System configuration and free energy analysis parameters

In [ ]:
# set thermal energy
TEMPERATURE_K: float = 300  # [K] # system temperature

# calculate free energy surface at the end of the simulation
cv = "dist_chain.z"
cv_offset = -0.3
start_frame_perc = 0.10
bandwidth = 0.075
min_val = 0.0
max_val = 2.5
n_grid = 500

# calculate free energy difference between the two states
lower_well = (0.5, 0.9)
upper_well = (1.6, 2.0)
n_timestep_plot = 250

# plotting parameters
ymax = 2


## Load data

In [ ]:
# create data pipeline
pipeline = DataPipeline(
    data_path_base=data_dir_base,
    temperature=TEMPERATURE_K,
    verbose=True
)


In [ ]:
# load data from Gromacs files and save as parquet
pipeline.save_plumed_colvar()

## Visualize data

### Convergence of the bias potential

In [ ]:
def fig_bias_dynamics(df: pd.DataFrame, fname: str) -> plt.Figure:
    fig = plt.figure(figsize=(22, 9))

    # CV: curve should show rapid sampling of the entire CV space
    ax1 = fig.add_subplot(241)
    ax1.set_xlabel("Time [ns]", labelpad=10)
    ax1.set_ylabel("$z$ [nm]", labelpad=10)
    ax1.set_title("Collective Variable", pad=10)
    ax1.scatter(df["time"]/1e3, df["dist_chain.z"], s=2, alpha=0.2)

    # OPES Bias: bias should be slowly increasing over time
    ax2 = fig.add_subplot(242)
    ax2.set_xlabel("Time [ns]", labelpad=10)
    ax2.set_ylabel("OPES Bias [kJ/mol]", labelpad=10)
    ax2.set_title("OPES Bias", pad=10)
    ax2.scatter(df["time"]/1e3, df["opes.bias"], s=2, alpha=0.2)

    # Lower wall bias: should hopefully be near zero for most of the simulation
    ax7 = fig.add_subplot(243)
    ax7.set_xlabel("Time [ns]", labelpad=10)
    ax7.set_ylabel("Bias [kJ/mol]", labelpad=10)
    ax7.set_title("Lower Wall Bias", pad=10)
    ax7.scatter(df["time"]/1e3, df["lower_wall.bias"], s=2, alpha=0.2)

    # Upper wall bias: should hopefully be near zero for most of the simulation 
    ax8 = fig.add_subplot(244)
    ax8.set_xlabel("Time [ns]", labelpad=10)
    ax8.set_ylabel("Bias [kJ/mol]", labelpad=10)
    ax8.set_title("Upper Wall Bias", pad=10)
    ax8.scatter(df["time"]/1e3, df["upper_wall.bias"], s=2, alpha=0.2)

    # OPES time constant: should increase and converge to a constant as the bias becomes quasi-static
    ax3 = fig.add_subplot(245)
    ax3.set_xlabel("Time [ns]", labelpad=10)
    ax3.set_ylabel("$c{(t)}$", labelpad=10)
    ax3.set_title("Quasi-Static Bias", pad=10)
    ax3.plot(df["time"]/1e3, df["opes.rct"], linewidth=3)

    # OPES normalization constant: starts from 1 an changes significantly when a new region of CV space is explored
    ax4 = fig.add_subplot(246)
    ax4.set_xlabel("Time [ns]", labelpad=10)
    ax4.set_ylabel("$Z_n$", labelpad=10)
    ax4.set_title("CV Exploration", pad=10)
    ax4.plot(df["time"]/1e3, df["opes.zed"], linewidth=3)

    # OPES number of effective samples: should be increasing over time with a fixed ratio to the number of samples
    ax5 = fig.add_subplot(247)
    ax5.set_xlabel("Time [ns]", labelpad=10)
    ax5.set_ylabel("$n_\mathrm{eff}$", labelpad=10)
    ax5.set_title("Effective Sample Size", pad=10)
    ax5.plot(df["time"]/1e3, df["opes.neff"], linewidth=3)

    # OPES number of compressed kernels: should increase and plateau
    ax6 = fig.add_subplot(248)
    ax6.set_xlabel("Time [ns]", labelpad=10)
    ax6.set_ylabel("$n_\mathrm{ker}$", labelpad=10)
    ax6.set_title("Compressed Kernels", pad=10)
    ax6.plot(df["time"]/1e3, df["opes.nker"], linewidth=3)

    # save figure
    fig.tight_layout()
    fig.savefig(f"{dir_fig}/{fname}.png", dpi=600, bbox_inches="tight")
    fig.savefig(f"{dir_fig}/{fname}.pdf", dpi=1200, bbox_inches="tight", transparent=True)

    return fig


In [ ]:
# plot bias dynamics for base replica
fname = "bias_dynamics"

if opes:
    fig = fig_bias_dynamics(data, fname)


In [ ]:
# plot bias dynamics for all replicas
if hrex and opes:
    for rep in range(8):
        fname = f"bias_dynamics_replica_{rep:02d}"
        fig = fig_bias_dynamics(all_data[rep], fname)
        plt.close(fig)


In [ ]:
# plot the collective variable for all replicas
fname = "cv_dynamics"

if hrex:
    fig = plt.figure(figsize=(20, 9))

    for i in range(8):
        ax = fig.add_subplot(2, 4, i+1)
        ax.set_xlabel("Time [ns]", labelpad=10)
        ax.set_ylabel("$z$ [nm]", labelpad=10)
        ax.set_title(f"Replica {i+1}", pad=10)
        ax.scatter(all_data[i]["time"]/1e3, all_data[i]["dist_chain.z"], s=1, alpha=0.2)

    fig.tight_layout()

    # save figure
    fig.savefig(f"{dir_fig}/{fname}.png", dpi=600, bbox_inches="tight")
    fig.savefig(f"{dir_fig}/{fname}.pdf", dpi=1200, bbox_inches="tight", transparent=True)


### Convergence of the free energy surface

In [ ]:
def fes_dist(
        x: np.ndarray, 
        weights: np.ndarray = None, 
        bandwidth: float = 0.1,
        x_grid: np.ndarray = None,
        min_val: float = None,
        max_val: float = None,
        n_grid: int = 300,
        d3_coord: bool = False,
    ) -> tuple[np.ndarray, np.ndarray]:
    """Calculate the free energy surface for a given distance collective variable.

    Parameters
    ----------
    x : np.ndarray
        Array of distances.
    weights : np.ndarray, optional
        Array of weights, by default None
    bandwidth : float, optional
        Bandwidth for kernel density estimation, by default 0.1
    x_grid : np.ndarray, optional
        Grid of distances to use for KDE, by default None
    min_val : float, optional
        Minimum value of distance, by default None
    max_val : float, optional
        Maximum value of distance, by default None
    n_grid : int, optional
        Number of grid points to use for KDE, by default 300
    d3_coord : bool, optional
        Whether the distance is a 3D coordinate, by default False
    
    Returns
    -------
    tuple[np.ndarray, np.ndarray]
        Tuple of arrays of distances and free energies.

    Raises
    ------
    ValueError
        If `min_val` is not less than `max_val`.
    """
    # input checking
    if not isinstance(x, np.ndarray):
        x = np.array(x)
    if weights is None:
        weights = np.ones_like(x)
    if not isinstance(weights, np.ndarray):
        weights = np.array(weights)
    
    # set kde domain
    if x_grid is None:
        if min_val is None:
            min_val = np.nanmin(x)
        if max_val is None:
            max_val = np.nanmax(x)
        if min_val >= max_val:
            raise ValueError(f"min_val ({min_val}) must be less than max_val ({max_val})")
        x_grid = np.linspace(min_val, max_val, n_grid)
    
    # calculate KDE of x weighted by weights
    kde = stats.gaussian_kde(x, weights=weights, bw_method=bandwidth)
    fes = - kde.logpdf(x_grid)

    # apply distance correction
    if d3_coord:
        fes += 2.0 * np.log(x_grid)

    # set minimum to zero
    fes -= np.nanmin(fes)

    return x_grid, fes


In [ ]:
def fe_diff(cv: np.ndarray, pmf: np.ndarray, lower_well: tuple[float, float], upper_well: tuple[float, float]) -> float:
    """
    Calculate the difference in free energy between the two wells.

    Parameters
    ----------
    cv : np.ndarray
        Array of collective variable values.
    pmf : np.ndarray
        Array of free energies as a function of collective variable, assumed to be unitless.
    lower_well : tuple[float, float]
        Tuple of (min, max) values for the lower well.
    upper_well : tuple[float, float]
        Tuple of (min, max) values for the upper well.
    
    Returns
    -------
    float
        Difference in free energy between the two wells. Unitless.
    """
    assert len(lower_well) == 2, "Lower well domain must have upper and lower bounds"
    assert len(upper_well) == 2, "Upper well domain must have upper and lower bounds"
    assert len(pmf) == len(cv), "PMF and CV must have the same size"

    # get indices of lower and upper wells
    lower_well_idx = np.where((cv > lower_well[0]) & (cv < lower_well[1]))
    upper_well_idx = np.where((cv > upper_well[0]) & (cv < upper_well[1]))

    # integrate boltzmann factors of wells to get probabilities
    boltzmann = np.exp(-pmf)
    prob_upper = integrate.simpson(boltzmann[upper_well_idx], x=cv[upper_well_idx])
    prob_lower = integrate.simpson(boltzmann[lower_well_idx], x=cv[lower_well_idx])

    # calculate free energy difference as log of ratio of probabilities
    delta_fe = - np.log(prob_lower / prob_upper)
    return delta_fe


### PMF estimate

In [ ]:
# extract CV and weight data from base replica
df = data.copy()
start_time = df["time"].max() * start_frame_perc
log.info(f"Initial time for analysis: {start_time/1e3} ns")
df_fes_in = df[df["time"] >= (df["time"].max() * start_frame_perc)].copy()
arr_cv = df_fes_in[cv].to_numpy()
arr_weight = df_fes_in["weight"].to_numpy()

log.debug(f"Number of samples: {len(arr_cv)}")

# calculate FES for the base replica
x_grid, fes = fes_dist(
    arr_cv,
    weights=arr_weight,
    bandwidth=bandwidth, 
    x_grid=np.linspace(min_val, max_val, n_grid),
    d3_coord=False,
)

# save data
df_fes = pd.DataFrame({"cv": x_grid, "fes": fes})
df_fes.to_csv(dir_data / "fes.csv", index=False)


In [ ]:
# plot free energy surface
fname = "pmf"

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title("Potential of Mean Force", pad=10)
ax.set_xlabel(r"$z$ [nm]", labelpad=10)
ax.set_ylabel(r"$\Delta F$ [$k_\mathrm{B}T$]", labelpad=10)
ax.set_ylim((0, ymax))
ax.plot(x_grid, fes, linewidth=3)

# save figure
fig.tight_layout()
fig.savefig(f"{dir_fig}/{fname}.png", dpi=600, bbox_inches="tight")
fig.savefig(f"{dir_fig}/{fname}.pdf", dpi=1200, bbox_inches="tight", transparent=True)


### Convergence of the free energy surface

In [ ]:
# calculate the free energy difference as a function of time
times = np.linspace(start_time + 1000, df["time"].max(), n_timestep_plot)
x_grid = np.linspace(min_val, max_val, n_grid)
pmfs = np.zeros((len(times), len(x_grid)))
delta_fes = np.zeros(len(times))

for i, t in tqdm(enumerate(times), total=len(times)):
    # calculate PMF estimate at time t
    cv_data = df_fes_in[df_fes_in["time"] <= t][cv].to_numpy()
    weight_data = df_fes_in[df_fes_in["time"] <= t]["weight"].to_numpy()
    x, pmf = fes_dist(
            cv_data,
            weights=weight_data,
            bandwidth=bandwidth, 
            x_grid=x_grid,
        )
    pmfs[i] = pmf

    # calculate free energy difference from PMF estimate
    delta_fes[i] = fe_diff(
        x,
        pmf,
        lower_well=lower_well,
        upper_well=upper_well,
    )

# save pmfs data as compressed npz with x_grid and times
np.savez_compressed(dir_data / "pmfs.npz", x_grid=x_grid, times=times, pmfs=pmfs)

# save delta FES data
df_delta_fes = pd.DataFrame({"time": times, "delta_fes": delta_fes})
df_delta_fes.to_csv(dir_data / "delta_fes.csv", index=False)


#### Free energy difference between the two states

In [ ]:
# plot the free energy difference as a function of time
fname = "pmf_diff_dynamics"

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel("Time [ns]")
ax.set_ylabel(r"$\Delta F$ [$k_\mathrm{B} T$]")
ax.set_title(
    f"$F({lower_well[0]} \leq r \leq {lower_well[1]}) - F({upper_well[0]} \leq r \leq {upper_well[1]})$", 
    y=1.03, 
    fontsize=20,
)

# add horizontal lines that are delta_fes[-1] +- 0.5
ax.axhline(delta_fes[-1] - 0.5, linestyle="--", color="k")
ax.axhline(delta_fes[-1] + 0.5, linestyle="--", color="k")
# fill in horizontal region between lines with opacity 0.2
ax.fill_between(
    [times[0]/1e3, times[-1]/1e3],
    delta_fes[-1] - 0.5, 
    delta_fes[-1] + 0.5, 
    alpha=0.1,
    color="k",
)
# plot delta F
ax.plot(times/1e3, delta_fes, linewidth=3)

# set ylim top to ymax if it is greater than ymax
if ymax is not None:
    if max(delta_fes) > ymax:
        ax.set_ylim(top=ymax)
    if min(delta_fes) < -ymax:
        ax.set_ylim(bottom=-ymax)
ax.set_ylim((-ymax, ymax))

# save figure
fig.tight_layout()
fig.savefig(f"{dir_fig}/{fname}.png", dpi=600, bbox_inches="tight")
fig.savefig(f"{dir_fig}/{fname}.pdf", dpi=1200, bbox_inches="tight", transparent=True)


#### Movie

In [ ]:
# plot the free energy surface as a function of time as a movie
fname = f"{dir_fig}/pmf_movie.mp4"
start_frame = int(start_frame_perc * len(data))

# setup figure
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel(r"$z$ [nm]", labelpad=10)
ax.set_ylabel(r"$\Delta F$ [$k_\mathrm{B}T$]", labelpad=10)
ax.set_ylim((0, ymax))
ax.set_title("Potential of Mean Force", pad=10)

# initialize plot elements
idx = 0
curve = ax.plot(x_grid, np.zeros_like(x_grid), linewidth=3)
text = ax.text(0.5, 0.9, f"$t_f = ${times[idx]/1e3:.2f} ns", transform=ax.transAxes, ha="center")

def animate(frame_num: int):
    global idx, times
    global curve, text

    curve[0].set_ydata(pmfs[idx])
    time = times[idx] / 1e3
    text.set_text(f"$t_f = $ {time:.1f} ns")
    idx += 1

    return curve, text


# animate
n_frames = len(times) - 1
log.debug(f"Number of frames: {n_frames}")
log.info(f"Creating animation")
anim = animation.FuncAnimation(
    fig, 
    animate, 
    blit=True,
    frames=n_frames,
)

# create tqdm progress bar
log.info(f"Saving animation as {fname}")
prog = tqdm(total=n_frames, desc="Saving animation", unit="frame", dynamic_ncols=True, colour="green")

# save animation as mp4 and output tqdm progress bar
anim.save(
    fname, 
    writer="ffmpeg", 
    fps=30, 
    dpi=300,
    progress_callback=lambda i, _: prog.update(1),
)
plt.close()

# display video in notebook
log.info(f"Displaying animation")
video = display.Video(fname, embed=True, width=700)
display.display(video)
